In [ ]:
# TO CHECK IF THE oLLAMA IS WROKING:

import requests
import json

url = "http://127.0.0.1:11434/v1/completions"

payload = {
    "model": "qwen2.5vl:7b",
    "prompt": "Hello!",
    "max_tokens": 100
}

response = requests.post(url, json=payload)

if response.status_code == 200:
    data = response.json()
    # The text is usually inside data["choices"][0]["text"]
    print("Response:", data["choices"][0]["text"])
else:
    print("Error:", response.status_code, response.text)


# ----------------- OLLAMA QWEN VL DOCUMENT EXTRACTOR -----------------

In [ ]:
import os
import json
import time
import base64
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import requests
from datetime import datetime
import dash
from dash import dcc, html, Input, Output, State, callback_context
import dash_bootstrap_components as dbc
import io


#-----------------------------------------------------------------------
# Configuration
#-----------------------------------------------------------------------
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "qwen2.5vl:7b"
OUTPUT_DIRECTORY = r"D:\MOhanaKrishnan\NEW_NAGA\OCR_API\DOCUMENTATION_OCR\JSON_OP"

# Ensure output directory exists
os.makedirs(OUTPUT_DIRECTORY, exist_ok=True)

class OllamaDocumentProcessor:
    def __init__(self):
        self.model_name = MODEL_NAME
        self.ollama_url = OLLAMA_URL
        
    def preprocess_image(self, image_content):
        """
        Optimized image preprocessing
        """
        try:
            # Convert bytes to PIL Image
            pil_img = Image.open(io.BytesIO(image_content))
            
            # Convert to RGB if necessary
            if pil_img.mode != 'RGB':
                pil_img = pil_img.convert('RGB')
            
            # Optimal resize - balance between quality and processing speed
            width, height = pil_img.size
            max_size = 1200
            if max(width, height) > max_size:
                ratio = max_size / max(width, height)
                new_size = (int(width * ratio), int(height * ratio))
                pil_img = pil_img.resize(new_size, Image.LANCZOS)
            elif max(width, height) < 600:
                # Upscale very small images
                ratio = 800 / max(width, height)
                new_size = (int(width * ratio), int(height * ratio))
                pil_img = pil_img.resize(new_size, Image.LANCZOS)
# -----------------------------------------------------------------------            
            # Enhanced image processing pipeline
# ----------------------------------------------------------------------- 
            # 1. Contrast enhancement
            enhancer = ImageEnhance.Contrast(pil_img)
            pil_img = enhancer.enhance(1.3)
            
            # 2. Sharpness enhancement
            enhancer = ImageEnhance.Sharpness(pil_img)
            pil_img = enhancer.enhance(1.2)
            
            # 3. Brightness adjustment
            enhancer = ImageEnhance.Brightness(pil_img)
            pil_img = enhancer.enhance(1.05)
            
            # 4. Noise reduction
            pil_img = pil_img.filter(ImageFilter.MedianFilter(size=3))
            
            # Convert to bytes
            img_byte_arr = io.BytesIO()
            pil_img.save(img_byte_arr, format='JPEG', quality=95)
            img_byte_arr = img_byte_arr.getvalue()
            
            # Convert to base64 for OLLAMA
            img_base64 = base64.b64encode(img_byte_arr).decode('utf-8')
            
            return img_base64
            
        except Exception as e:
            print(f"Error in image preprocessing: {str(e)}")
            return None

    def detect_document_type_and_extract(self, image_content, filename):
        """
        Process document using OLLAMA API
        """
        start_time = time.time()
        
        try:
            # Preprocess image
            processed_image_b64 = self.preprocess_image(image_content)
            if processed_image_b64 is None:
                return {"error": "Could not process image", "document_type": "unknown"}
            
            # Universal prompt for document detection and extraction
# -----------------------------------------------------------------------
            # PROMPT
# -----------------------------------------------------------------------
            prompt = """
            Analyze this image and determine what type of Indian government document it is. Then extract all relevant details.

            First, identify the document type:
            1. PAN Card (Permanent Account Number)
            2. RC Book (Registration Certificate for vehicles) 
            3. Driving License (DL)
            4. AADHAAR Card (Unique Identification)
            5. Other/Unknown

            Based on the document type, extract the following details in JSON format:

            FOR PAN CARD:
            {
                "document_type": "PAN",
                "is_genuine": true/false,
                "pan_number": "extracted PAN number",
                "name": "extracted name",
                "fathers_name": "extracted father's name",
                "date_of_birth": "DD/MM/YYYY",
                "signature_present": true/false,
                "photo_present": true/false,
                "confidence_score": 0.95
            }

            FOR RC BOOK:
            {
                "document_type": "RC",
                "is_genuine": true/false,
                "registration_number": "extracted registration number",
                "date_of_registration": "DD/MM/YYYY",
                "registration_valid_till": "DD/MM/YYYY",
                "owner_serial_number": "extracted owner serial number",
                "chassis_number": "extracted chassis number",
                "engine_number": "extracted engine number", 
                "owner_name": "extracted owner name",
                "son_daughter_wife_of": "extracted father/husband name",
                "address": "extracted address",
                "fuel_type": "extracted fuel type",
                "vehicle_class": "extracted vehicle class",
                "state": "extracted state",
                "confidence_score": 0.95
            }

            FOR DRIVING LICENSE:
            {
                "document_type": "DL",
                "is_genuine": true/false,
                "side": "front/back/complete",
                "dl_no": "extracted DL number",
                "name": "extracted name",
                "son_daughter_wife_of": "extracted father/husband name",
                "date_of_birth": "DD/MM/YYYY",
                "date_of_issue": "DD/MM/YYYY", 
                "valid_till": "DD/MM/YYYY",
                "blood_group": "extracted blood group",
                "address": "extracted address",
                "state": "extracted state",
                "confidence_score": 0.95
            }

            FOR AADHAAR CARD:
            {
                "document_type": "AADHAAR",
                "is_genuine": true/false,
                "aadhaar_number": "extracted AADHAAR number (12 digits)",
                "name": "extracted name",
                "date_of_birth": "DD/MM/YYYY",
                "year_of_birth": "YYYY",
                "gender": "Male/Female/Other",
                "address": "extracted address",
                "photo_present": true/false,
                "confidence_score": 0.95
            }

            FOR OTHER/UNKNOWN DOCUMENTS:
            {
                "document_type": "UNKNOWN",
                "is_genuine": false,
                "reason": "Not a recognized Indian government document",
                "confidence_score": 0.95
            }

            Be very accurate with text extraction. Look for official logos, formatting, and standard document elements.
            Only return valid JSON, no additional text.
            """
            
            # Prepare OLLAMA request
            payload = {
                "model": self.model_name,
                "prompt": prompt,
                "images": [processed_image_b64],
                "stream": False
            }
            
            # Send request to OLLAMA
            response = requests.post(self.ollama_url, json=payload)
            
            if response.status_code != 200:
                return {
                    "document_type": "ERROR",
                    "is_genuine": False,
                    "reason": f"OLLAMA API error: {response.status_code}",
                    "error": response.text
                }
            
            # Parse OLLAMA response
            ollama_response = response.json()
            response_text = ollama_response.get('response', '')
            
            # Clean and parse JSON response
            try:
                response_text = response_text.strip()
                
                # Clean response
                if response_text.startswith('```json'):
                    response_text = response_text.replace('```json', '').replace('```', '').strip()
                elif response_text.startswith('```'):
                    response_text = response_text.replace('```', '').strip()
                
                processing_time = time.time() - start_time
                result = json.loads(response_text)
                
                # Add metadata
                result["processed_timestamp"] = datetime.now().isoformat()
                result["processing_time_seconds"] = round(processing_time, 2)
                result["filename"] = filename
                result["model_used"] = self.model_name
                
                return result
                
            except json.JSONDecodeError as e:
                return {
                    "document_type": "ERROR",
                    "is_genuine": False,
                    "reason": "Failed to parse OLLAMA response",
                    "raw_response": response_text,
                    "error": str(e)
                }
                
        except Exception as e:
            return {
                "document_type": "ERROR",
                "is_genuine": False,
                "reason": f"Processing error: {str(e)}",
                "error": str(e)
            }
# -----------------------------------------------------------------------
# Initialize the processor
# -----------------------------------------------------------------------
processor = OllamaDocumentProcessor()

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'OLLAMA Document Extractor'

# -----------------------------------------------------------------------
# Custom CSS and JavaScript
# -----------------------------------------------------------------------

custom_css = """
<style>
    /* Professional color palette using Viridis/Teal theme */
    :root {
        --primary-color: #1f77b4;
        --secondary-color: #2ca02c;
        --accent-color: #ff7f0e;
        --danger-color: #d62728;
        --teal-dark: #004d40;
        --teal-medium: #00695c;
        --teal-light: #4db6ac;
        --viridis-purple: #440154;
        --viridis-blue: #3b528b;
        --viridis-green: #21908c;
        --viridis-yellow: #fde725;
    }

    /* Main container styling */
    .main-container {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        min-height: 100vh;
        padding: 20px 0;
    }

    /* Title card styling */
    .title-card {
        background: linear-gradient(135deg, var(--teal-dark), var(--teal-medium));
        color: white;
        border-radius: 15px;
        box-shadow: 0 10px 30px rgba(0,0,0,0.2);
        margin-bottom: 30px;
        animation: slideInDown 0.8s ease-out;
    }

    .title-card h1 {
        font-size: 2.5rem;
        font-weight: 700;
        margin-bottom: 10px;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
    }

    .subtitle {
        font-size: 1.1rem;
        opacity: 0.9;
        font-weight: 300;
    }

    /* Stats container card */
    .stats-container {
        background: rgba(255, 255, 255, 0.95);
        border-radius: 15px;
        box-shadow: 0 8px 25px rgba(0,0,0,0.1);
        padding: 20px;
        margin-bottom: 25px;
        animation: fadeInUp 0.8s ease-out 0.2s both;
    }

    .stats-title {
        text-align: center;
        margin-bottom: 20px;
        color: var(--teal-dark);
        font-weight: 600;
    }

    /* Individual stat cards */
    .stat-card {
        transition: all 0.3s ease;
        border-radius: 12px;
        overflow: hidden;
        position: relative;
    }

    .stat-card:hover {
        transform: translateY(-5px);
        box-shadow: 0 15px 35px rgba(0,0,0,0.1);
    }

    .stat-card.aadhaar {
        background: linear-gradient(135deg, var(--viridis-purple), var(--viridis-blue));
        color: white;
    }

    .stat-card.pan {
        background: linear-gradient(135deg, var(--viridis-green), var(--teal-light));
        color: white;
    }

    .stat-card.dl {
        background: linear-gradient(135deg, var(--primary-color), #5dade2);
        color: white;
    }

    .stat-card.rc {
        background: linear-gradient(135deg, var(--accent-color), #f7dc6f);
        color: white;
    }

    /* Loading spinner */
    .loading-container {
        display: flex;
        justify-content: center;
        align-items: center;
        padding: 40px;
    }

    .spinner {
        width: 60px;
        height: 60px;
        border: 4px solid rgba(255, 255, 255, 0.3);
        border-top: 4px solid var(--teal-medium);
        border-radius: 50%;
        animation: spin 1s linear infinite;
    }

    .processing-text {
        margin-left: 20px;
        font-size: 1.1rem;
        color: var(--teal-dark);
        font-weight: 500;
    }

    /* Card animations */
    @keyframes slideInDown {
        from {
            opacity: 0;
            transform: translateY(-50px);
        }
        to {
            opacity: 1;
            transform: translateY(0);
        }
    }

    @keyframes fadeInUp {
        from {
            opacity: 0;
            transform: translateY(30px);
        }
        to {
            opacity: 1;
            transform: translateY(0);
        }
    }

    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }

    @keyframes pulse {
        0%, 100% { opacity: 1; }
        50% { opacity: 0.7; }
    }

    .pulse-animation {
        animation: pulse 2s infinite;
    }

    /* Upload area styling */
    .upload-area {
        background: linear-gradient(135deg, rgba(255,255,255,0.9), rgba(255,255,255,0.7));
        border: 2px dashed var(--teal-medium);
        border-radius: 12px;
        transition: all 0.3s ease;
    }

    .upload-area:hover {
        border-color: var(--teal-dark);
        background: rgba(255,255,255,1);
        transform: translateY(-2px);
    }

    /* Professional button styling */
    .btn-process {
        background: linear-gradient(135deg, var(--teal-medium), var(--teal-dark));
        border: none;
        border-radius: 8px;
        font-weight: 600;
        text-transform: uppercase;
        letter-spacing: 1px;
        transition: all 0.3s ease;
    }

    .btn-process:hover {
        background: linear-gradient(135deg, var(--teal-dark), var(--viridis-purple));
        transform: translateY(-2px);
        box-shadow: 0 8px 20px rgba(0,0,0,0.2);
    }

    /* Results card styling */
    .results-card {
        background: rgba(255, 255, 255, 0.95);
        border-radius: 15px;
        box-shadow: 0 8px 25px rgba(0,0,0,0.1);
        animation: fadeInUp 0.8s ease-out;
    }
</style>

<script>
    // Professional loading and animation effects
    window.dash_clientside = Object.assign({}, window.dash_clientside, {
        clientside: {
            add_loading_animation: function(n_clicks) {
                if (n_clicks) {
                    // Add loading spinner
                    setTimeout(function() {
                        const statusDiv = document.querySelector('#processing-status');
                        if (statusDiv) {
                            statusDiv.innerHTML = `
                                <div class="loading-container">
                                    <div class="spinner"></div>
                                    <div class="processing-text pulse-animation">
                                        🤖 AI Model Processing Document...
                                    </div>
                                </div>
                            `;
                        }
                    }, 100);
                }
                return '';
            },
            
            animate_stats: function(aadhaar, pan, dl, rc) {
                // Animate counter updates
                setTimeout(function() {
                    const cards = document.querySelectorAll('.stat-card');
                    cards.forEach(card => {
                        card.style.animation = 'pulse 0.6s ease-in-out';
                    });
                }, 200);
                return '';
            }
        }
    });
</script>
"""

# -----------------------------------------------------------------------
# Custom HTML template to include custom CSS and JS
# -----------------------------------------------------------------------
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        ''' + custom_css + '''
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# -----------------------------------------------------------------------
# App layout
# -----------------------------------------------------------------------

app.layout = html.Div(className="main-container", children=[
    dbc.Container([
        # Title Card
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H1("🔍 Document Extractor", className="text-center mb-3"),
                        html.P("Powered By OLLAMA — Qwen 2.5VL", className="text-center subtitle mb-0")
                    ], className="py-4")
                ], className="title-card")
            ])
        ], className="mb-4"),
        
        # Stats Container Card
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4("📊 Document Processing Statistics", className="stats-title"),
                        dbc.Row([
                            dbc.Col([
                                dbc.Card([
                                    dbc.CardBody([
                                        html.H6("🆔 AADHAAR", className="card-title text-center mb-2"),
                                        html.H3(id="aadhaar-count", children="0", className="text-center mb-2"),
                                        html.P("Processed", className="text-center mb-0 small")
                                    ], className="py-3")
                                ], className="stat-card aadhaar")
                            ], width=3, className="mb-3"),
                            
                            dbc.Col([
                                dbc.Card([
                                    dbc.CardBody([
                                        html.H6("🏷️ PAN", className="card-title text-center mb-2"),
                                        html.H3(id="pan-count", children="0", className="text-center mb-2"),
                                        html.P("Processed", className="text-center mb-0 small")
                                    ], className="py-3")
                                ], className="stat-card pan")
                            ], width=3, className="mb-3"),
                            
                            dbc.Col([
                                dbc.Card([
                                    dbc.CardBody([
                                        html.H6("🚘 Driving License", className="card-title text-center mb-2"),
                                        html.H3(id="dl-count", children="0", className="text-center mb-2"),
                                        html.P("Processed", className="text-center mb-0 small")
                                    ], className="py-3")
                                ], className="stat-card dl")
                            ], width=3, className="mb-3"),
                            
                            dbc.Col([
                                dbc.Card([
                                    dbc.CardBody([
                                        html.H6("🚗 RC Book", className="card-title text-center mb-2"),
                                        html.H3(id="rc-count", children="0", className="text-center mb-2"),
                                        html.P("Processed", className="text-center mb-0 small")
                                    ], className="py-3")
                                ], className="stat-card rc")
                            ], width=3, className="mb-3")
                        ])
                    ])
                ], className="stats-container")
            ])
        ], className="mb-4"),
        
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader(html.H4("📁 Upload Document")),
                    dbc.CardBody([
                        dcc.Upload(
                            id='upload-image',
                            children=html.Div([
                                html.I(className="fas fa-cloud-upload-alt", style={"fontSize": "48px", "color": "#00695c", "marginBottom": "10px"}),
                                html.Br(),
                                'Drag and Drop or ',
                                html.A('Select Files', style={"color": "#00695c", "fontWeight": "bold"})
                            ]),
                            className='upload-area',
                            style={
                                'width': '100%',
                                'height': '120px',
                                'lineHeight': '120px',
                                'textAlign': 'center',
                                'margin': '10px 0'
                            },
                            multiple=False,
                            accept='image/*'
                        ),
                        html.Div(id='upload-status', className="mt-3"),
                        html.Hr(),
                        dbc.Button(
                            "🚀 Process Document", 
                            id="process-button", 
                            className="btn-process w-100", 
                            size="lg",
                            disabled=True
                        )
                    ])
                ], className="results-card")
            ], width=12)
        ], className="mb-4"),
        
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader(html.H4("⚡ Processing Status")),
                    dbc.CardBody([
                        html.Div(id="processing-status", children="Upload a document to begin processing..."),
                        html.Div(id="processing-progress", className="mt-3")
                    ])
                ], className="results-card")
            ])
        ], className="mb-4"),
        
        dbc.Row([
            dbc.Col([
                html.Div(id="results-display")
            ])
        ]),
        
        # Hidden components for data storage
        dcc.Store(id="uploaded-file-data"),
        dcc.Store(id="results-store"),
        dcc.Store(id="document-counts", data={"AADHAAR": 0, "PAN": 0, "DL": 0, "RC": 0}),
        
        # Clientside callback triggers
        html.Div(id="loading-trigger", style={"display": "none"}),
        html.Div(id="stats-trigger", style={"display": "none"})
        
    ], fluid=True)
])

# -----------------------------------------------------------------------
# APP CALLBACKS
# -----------------------------------------------------------------------

# Upload callback
@app.callback(
    [Output("uploaded-file-data", "data"),
     Output("upload-status", "children"),
     Output("process-button", "disabled")],
    [Input("upload-image", "contents")],
    [State("upload-image", "filename")]
)
def handle_upload(contents, filename):
    if contents is None:
        return None, "", True
    
    try:
        # Parse the uploaded file
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        
        # Store file data
        file_data = {
            "content": base64.b64encode(decoded).decode(),
            "filename": filename
        }
        
        status = dbc.Alert([
            html.H6("✅ File Uploaded Successfully!", className="alert-heading"),
            html.P(f"📄 Filename: {filename}"),
            html.P(f"📏 Size: {len(decoded)} bytes"),
            html.P("Click 'Process Document' to analyze the image.")
        ], color="success")
        
        return file_data, status, False
        
    except Exception as e:
        error_status = dbc.Alert(f"❌ Upload Error: {str(e)}", color="danger")
        return None, error_status, True

# Process callback
@app.callback(
    [Output("results-store", "data"),
     Output("processing-status", "children"),
     Output("processing-progress", "children"),
     Output("document-counts", "data"),
     Output("aadhaar-count", "children"),
     Output("pan-count", "children"),
     Output("dl-count", "children"),
     Output("rc-count", "children")],
    [Input("process-button", "n_clicks")],
    [State("uploaded-file-data", "data"),
     State("document-counts", "data")]
)
def process_document(n_clicks, file_data, current_counts):
    if n_clicks is None or file_data is None:
        return None, "Upload a document to begin...", "", current_counts, str(current_counts.get("AADHAAR", 0)), str(current_counts.get("PAN", 0)), str(current_counts.get("DL", 0)), str(current_counts.get("RC", 0))
    
    try:
        # Show processing status
        processing_status = dbc.Alert([
            html.H6("⏳ Processing Document...", className="alert-heading"),
            html.P("Analyzing document with OLLAMA qwen2.5vl:7b model...")
        ], color="info")
        
        # Decode file content
        file_content = base64.b64decode(file_data["content"])
        filename = file_data["filename"]
        
        # Process the document
        result = processor.detect_document_type_and_extract(file_content, filename)
        
        # Save result to JSON file
        json_filename = f"{os.path.splitext(filename)[0]}_extracted.json"
        json_path = os.path.join(OUTPUT_DIRECTORY, json_filename)
        
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(result, f, indent=2, ensure_ascii=False)
        
        # Update status
        success_status = dbc.Alert([
            html.H6("✅ Processing Complete!", className="alert-heading"),
            html.P(f"📄 Document processed: {filename}"),
            html.P(f"💾 Results saved to: {json_filename}"),
            html.P(f"⏱️ Processing time: {result.get('processing_time_seconds', 0):.2f}s")
        ], color="success")
        
        # Update document counts
        doc_type = result.get('document_type', 'UNKNOWN')
        is_genuine = result.get('is_genuine', False)

        if doc_type in ['AADHAAR', 'PAN', 'DL', 'RC'] and is_genuine:
            current_counts[doc_type] = current_counts.get(doc_type, 0) + 1

        return result, success_status, "", current_counts, str(current_counts.get("AADHAAR", 0)), str(current_counts.get("PAN", 0)), str(current_counts.get("DL", 0)), str(current_counts.get("RC", 0))
        
    except Exception as e:
        error_status = dbc.Alert(f"❌ Processing Error: {str(e)}", color="danger")
        return None, error_status, "", current_counts, str(current_counts.get("AADHAAR", 0)), str(current_counts.get("PAN", 0)), str(current_counts.get("DL", 0)), str(current_counts.get("RC", 0))

# Display results callback
@app.callback(
    Output("results-display", "children"),
    [Input("results-store", "data")]
)
def display_results(result):
    if not result:
        return ""
    
    filename = result.get('filename', 'Unknown')
    doc_type = result.get('document_type', 'UNKNOWN')
    is_genuine = result.get('is_genuine', False)
    
    # Determine card color and icon
    if doc_type == "PAN" and is_genuine:
        color = "success"
        icon = "🏷️"
        header = f"{icon} PAN Card Analysis - {filename}"
    elif doc_type == "RC" and is_genuine:
        color = "info"
        icon = "🚗"
        header = f"{icon} RC Book Analysis - {filename}"
    elif doc_type == "DL" and is_genuine:
        color = "primary"
        icon = "🚘"
        header = f"{icon} Driving License Analysis - {filename}"
    elif doc_type == "AADHAAR" and is_genuine:
        color = "secondary"
        icon = "🆔"
        header = f"{icon} AADHAAR Card Analysis - {filename}"
    else:
        color = "warning"
        icon = "❓"
        header = f"{icon} Document Analysis - {filename}"
    
    # Create card content based on document type
    card_content = []
    
    if doc_type == "PAN" and is_genuine:
        card_content = [
            html.P([html.Strong("PAN Number: "), result.get("pan_number", "N/A")]),
            html.P([html.Strong("Name: "), result.get("name", "N/A")]),
            html.P([html.Strong("Father's Name: "), result.get("fathers_name", "N/A")]),
            html.P([html.Strong("Date of Birth: "), result.get("date_of_birth", "N/A")]),
            html.P([html.Strong("Photo Present: "), "Yes" if result.get("photo_present") else "No"]),
            html.P([html.Strong("Signature Present: "), "Yes" if result.get("signature_present") else "No"])
        ]
    elif doc_type == "RC" and is_genuine:
        card_content = [
            html.P([html.Strong("Registration Number: "), result.get("registration_number", "N/A")]),
            html.P([html.Strong("Owner Name: "), result.get("owner_name", "N/A")]),
            html.P([html.Strong("Vehicle Class: "), result.get("vehicle_class", "N/A")]),
            html.P([html.Strong("Fuel Type: "), result.get("fuel_type", "N/A")]),
            html.P([html.Strong("Registration Date: "), result.get("date_of_registration", "N/A")]),
            html.P([html.Strong("Valid Till: "), result.get("registration_valid_till", "N/A")]),
            html.P([html.Strong("Chassis Number: "), result.get("chassis_number", "N/A")]),
            html.P([html.Strong("Engine Number: "), result.get("engine_number", "N/A")])
        ]
    elif doc_type == "DL" and is_genuine:
        card_content = [
            html.P([html.Strong("DL Number: "), result.get("dl_no", "N/A")]),
            html.P([html.Strong("Name: "), result.get("name", "N/A")]),
            html.P([html.Strong("Father/Husband Name: "), result.get("son_daughter_wife_of", "N/A")]),
            html.P([html.Strong("Date of Birth: "), result.get("date_of_birth", "N/A")]),
            html.P([html.Strong("Date of Issue: "), result.get("date_of_issue", "N/A")]),
            html.P([html.Strong("Valid Till: "), result.get("valid_till", "N/A")]),
            html.P([html.Strong("Blood Group: "), result.get("blood_group", "N/A")]),
            html.P([html.Strong("Address: "), result.get("address", "N/A")])
        ]
    elif doc_type == "AADHAAR" and is_genuine:
        card_content = [
            html.P([html.Strong("AADHAAR Number: "), result.get("aadhaar_number", "N/A")]),
            html.P([html.Strong("Name: "), result.get("name", "N/A")]),
            html.P([html.Strong("Date of Birth: "), result.get("date_of_birth", "N/A")]),
            html.P([html.Strong("Year of Birth: "), result.get("year_of_birth", "N/A")]),
            html.P([html.Strong("Gender: "), result.get("gender", "N/A")]),
            html.P([html.Strong("Address: "), result.get("address", "N/A")]),
            html.P([html.Strong("Photo Present: "), "Yes" if result.get("photo_present") else "No"])
        ]
    else:
        card_content = [
            html.P([html.Strong("Status: "), "Error" if doc_type == "ERROR" else "Unknown Document"]),
            html.P([html.Strong("Reason: "), result.get("reason", "Unknown error")]),
            html.P([html.Strong("Document Type: "), doc_type])
        ]
    
    """# Add processing metadata
    card_content.extend([
        html.Hr(),
        html.P([html.Strong("Processing Time: "), f"{result.get('processing_time_seconds', 0):.2f}s"]),
        html.P([html.Strong("Confidence: "), f"{result.get('confidence_score', 0):.2%}"]),
        html.P([html.Strong("Model Used: "), result.get('model_used', 'N/A')])
    ])"""
    
    result_card = dbc.Card([
        dbc.CardHeader(html.H5(header, style={"margin": 0})),
        dbc.CardBody(card_content)
    ], style={
        "background": f"linear-gradient(135deg, rgba(255,255,255,0.9), rgba(255,255,255,0.7))",
        "border": f"2px solid {color}",
        "borderRadius": "12px",
        "boxShadow": "0 8px 25px rgba(0,0,0,0.1)"
    }, className="mb-3")
    
    return dbc.Card([
        dbc.CardHeader(html.H4("📄 Extraction Results")),
        dbc.CardBody([result_card])
    ], className="mb-4")

# -----------------------------------------------------------------------
# CLIENTSIDE CALLBACKS
# -----------------------------------------------------------------------

# Clientside callback for loading animation
app.clientside_callback(
    "window.dash_clientside.clientside.add_loading_animation",
    Output("loading-trigger", "children"),
    [Input("process-button", "n_clicks")]
)

# Clientside callback for stats animation
app.clientside_callback(
    "window.dash_clientside.clientside.animate_stats",
    Output("stats-trigger", "children"),
    [Input("aadhaar-count", "children"),
     Input("pan-count", "children"),
     Input("dl-count", "children"),
     Input("rc-count", "children")]
)


# -----------------------------------------------------------------------
# Run the app
# -----------------------------------------------------------------------

if __name__ == "__main__":
    print("Starting OLLAMA Document Extractor...")
    print("Make sure OLLAMA is running at http://localhost:11434")
    print("Open http://127.0.0.1:7050 in your browser")
    
    app.run(debug=True, port=7050)

Starting OLLAMA Document Extractor...
Make sure OLLAMA is running at http://localhost:11434
Open http://127.0.0.1:7050 in your browser
